In [2]:
sc.textFile('data/coupon150720.csv').first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [6]:
from time import strptime
from time import mktime

In [4]:
d1 = strptime('150904','%y%m%d')

In [5]:
d2 = strptime('150623','%y%m%d')

In [7]:
mktime(d1)

1441317600.0

In [9]:
(mktime(d1) - mktime(d2)) / (3600*24)

73.0

In [10]:
# 73 días de diferencia

In [11]:
sc.textFile('data/transm150720.csv').first()

u'79062005698500,TKTT,30,150719,FR,0.0,EUR,T,T,141025,PARA127A8,0.0,EUR,   ,EX,,150719,0.0'

In [12]:
#lo lógico sería hacer la conversión de fechas al final
def getFechaCompra(v):
    e = v.split(",")
    ticket = e[0]
    fecha_compra = mktime(strptime(e[9],'%y%m%d'))
    return (ticket, fecha_compra)

In [15]:
def getFechaCompra(v):
    e = v.split(",")
    ticket = e[0]
    fecha_compra = e[9]
    return (ticket, fecha_compra)

In [16]:
sc.textFile('data/transm150720.csv').map(getFechaCompra).take(2)

[(u'79062005698500', u'141025'), (u'79062005924069', u'150719')]

In [17]:
tc = sc.textFile('data/transm150720.csv').map(getFechaCompra)

In [20]:
def getFechaSalida(v):
    e = v.split(",")
    ticket = e[0]
    fecha_salida = e[12]
    return (ticket, fecha_salida)

In [21]:
sc.textFile('data/coupon150720.csv').map(getFechaSalida).take(2)

[(u'79062005698500', u'150904'), (u'79062005698500', u'150905')]

In [23]:
fechas_salidas = sc.textFile('data/coupon150720.csv').map(getFechaSalida)

In [24]:
fechas_salidas.groupByKey().take(2)

[(u'79062005558463',
  <pyspark.resultiterable.ResultIterable at 0x7fb3670aa610>),
 (u'79065668432713',
  <pyspark.resultiterable.ResultIterable at 0x7fb3670aafd0>)]

In [33]:
#sol
def fechaMenor(ds):
    segs = [mktime(strptime(d, '%y%m%d')) for d in ds]
    return min(segs)

In [32]:
tc.cogroup(fechas_salidas).take(2)

[(u'79062005994272',
  (<pyspark.resultiterable.ResultIterable at 0x7fb367069810>,
   <pyspark.resultiterable.ResultIterable at 0x7fb367069850>)),
 (u'79065668614400',
  (<pyspark.resultiterable.ResultIterable at 0x7fb3670698d0>,
   <pyspark.resultiterable.ResultIterable at 0x7fb367069910>))]

In [34]:
tc.cogroup(fechas_salidas).mapValues(lambda x: fechaMenor(x[1])).take(2)

[(u'79062005994272', 1437343200.0), (u'79065668614400', 1437688800.0)]

In [36]:
def diasDeDiferencia(fechaCompraStr, fechaSalidaSegs):
    fechaCompraStruct = strptime(fechaCompraStr,'%y%m%d')
    fechaCompraSegs = mktime(fechaCompraStruct)
    return (fechaSalidaSegs-fechaCompraSegs)/3600*24

In [43]:
fts = tc.filter(lambda x: x[1] != '')
fcs = fechas_salidas.filter(lambda x: x[1] != '')

In [ ]:
fts.cogroup(fcs).mapValues(lambda x: diasDeDiferencia(fechaMenor(x[1]), fechaMenor(x[0]))).take(2)

In [ ]:
r =

In [ ]:
r.mapValues()